# Extract TMRCA of T15

Import necessary libraries

In [1]:
import pandas as pd
from dendropy import Tree

Load in the metadata for each sequence

In [2]:
metadata = pd.read_csv( "../../data/vc_metadata_workshop.csv", parse_dates=["collection_date"] )

Extract the members of each putative introduction. The `lineage_members.txt` file was manually generated by observing the consensus tree.

In [3]:
members = dict()
with open( "lineage_members.txt", "r" ) as members_file:
	for line in members_file:
		if line.strip() == "":
			continue
		elif line.startswith( ">" ):
			group = line.strip().replace( ">", "" )
			members[group] = []
		else:
			members[group].append( line.strip() )

for i in members:
	print( f"{i} - {len( members[i] )} members")

Small - 8 members
Large - 322 members


Here we iterate through the posterior distribution and estimate the TMRCA of the two putative T15 introductions. For each tree, we use the lineage members to identify the MRCA, the height of this node corresponds to the TMRCA. 

BEAST encodes dates relatively, that is each node records its date in decimal years since the root of the tree. Because the root age is allowed to change, we have to calculate the date of the root for each tree. We calculate this by identifying the most recent sampling date (that is the sequence with the latest collection date), and subtracting from it the maximum distance between root and tip in the tree. This yields the value `root_age` which we can add to each node's age to get a date in decimal years.

In [5]:
dates = {
	"tree_id" : [],
	"group" : [],
	"date" : []
}

trees = Tree.yield_from_files( files=["../../beast_analyses/2024-04-29_constant_relaxed.down.trees"], schema="nexus", preserve_underscores=True )
burnin = 300

for tree_idx, tree in enumerate( trees ):
	if tree_idx == 0:
		taxa = [i.label for i in tree.taxon_namespace]
		tree_md = metadata.loc[metadata["taxa"].isin(taxa)]
		mrsd = tree_md["collection_date"].max().year + (tree_md["collection_date"].max().dayofyear / 365.25 )
	if tree_idx < burnin:
		continue
	root_age = mrsd - tree.seed_node.distance_from_tip()
	
	for group, mems in members.items():
		mrca = tree.mrca( taxon_labels=mems )
		date = root_age + mrca.distance_from_root()
		dates["group"].append( group )
		dates["date"].append( date )
		dates["tree_id"].append( tree_idx )

We save the estimated TMRCAs to file so that we can access them later.

In [6]:
dates_df = pd.DataFrame( dates )
dates_df.to_csv( "tmrca_t15.csv", index=False )
dates_df.head()

,tree_id,group,date
0,300,Small,2022.490936
1,300,Large,2019.556459
2,301,Small,2022.937029
3,301,Large,2019.913291
4,302,Small,2022.533689
